In [ ]:
import pandas as pd
import numpy as np
import math
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier


def logis_reg(X_train,y_train,X_test,y_test):
    model = LogisticRegression()
    # define evaluation
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    # define search space
    space = dict()
    space['solver'] = ['newton-cg', 'lbfgs', 'liblinear']
    space['penalty'] = ['none', 'l1', 'l2', 'elasticnet']
    space['C'] = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
    # define search
    search = GridSearchCV(model, space, scoring='accuracy', n_jobs=-1, cv=cv)
    search.fit(X_train,y_train)
    predicted=search.predict(X_test)
    a=accuracy_score(y_test, predicted)
    p=precision_score(y_test,predicted,average="macro")
    r=recall_score(y_test,predicted,average="macro")
    f=f1_score(y_test,predicted,average="macro")
    return a,p,r,f

def rand_forest(X_train,y_train,X_test,y_test):
    # Define Parameters
    max_depth=[2,3,5,10,15,20]
    n_estimators = [2,5,10,15,18,20,22]
    param_grid = dict(max_depth=max_depth, n_estimators=n_estimators)

    # Build the grid search
    dfrst = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)
    grid = GridSearchCV(estimator=dfrst, param_grid=param_grid, cv = 5)
    grid.fit(X_train, y_train)
    predicted=grid.predict(X_test)
    a=accuracy_score(y_test, predicted)
    p=precision_score(y_test,predicted,average="macro")
    r=recall_score(y_test,predicted,average="macro")
    f=f1_score(y_test,predicted,average="macro")
    return a,p,r,f


def decis_tree(X_train,y_train,X_test,y_test):
    dt = DecisionTreeClassifier(random_state=42)
    params = {
        'max_depth': [2, 3, 5, 10, 20],
        'min_samples_leaf': [5, 10, 20, 50, 100],
        'criterion': ["gini", "entropy"]}
    grid_search = GridSearchCV(estimator=dt, 
                               param_grid=params, 
                               cv=5, n_jobs=-1, verbose=1)
    grid_search.fit(X_train, y_train)
    predicted=grid_search.predict(X_test)
    a=accuracy_score(y_test, predicted)
    p=precision_score(y_test,predicted,average="macro")
    r=recall_score(y_test,predicted,average="macro")
    f=f1_score(y_test,predicted,average="macro")
    return a,p,r,f



def navie_bayes(X_train,y_train,X_test,y_test):
    gnb = GaussianNB()
    gnb.fit(X_train, y_train)
    predicted=gnb.predict(X_test)
    a=accuracy_score(y_test, predicted)
    p=precision_score(y_test,predicted,average="macro")
    r=recall_score(y_test,predicted,average="macro")
    f=f1_score(y_test,predicted,average="macro")
    return a,p,r,f


def knn(X_train,y_train,X_test,y_test):
    knn = KNeighborsClassifier()
    grid_params = { 'n_neighbors' : [2,3,4,5,7,9,11,13,15],
                   'weights' : ['uniform','distance'],
                   'metric' : ['minkowski','euclidean','manhattan']}
    gs = GridSearchCV(KNeighborsClassifier(), grid_params, verbose = 1, cv=3, n_jobs = -1)
    g_res = gs.fit(X_train, y_train)
    predicted=g_res.predict(X_test)
    a=accuracy_score(y_test, predicted)
    p=precision_score(y_test,predicted,average="macro")
    r=recall_score(y_test,predicted,average="macro")
    f=f1_score(y_test,predicted,average="macro")
    return a,p,r,f


def svm(X_train,y_train,X_test,y_test):
    model = SVC()
    param_grid = {'C': [0.1, 1, 10, 100, 1000], 
                  'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                  'kernel': ['rbf','poly','linear']} 
    grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)
    grid.fit(X_train, y_train)
    predicted=grid.predict(X_test)
    a=accuracy_score(y_test, predicted)
    p=precision_score(y_test,predicted,average="macro")
    r=recall_score(y_test,predicted,average="macro")
    f=f1_score(y_test,predicted,average="macro")
    return a,p,r,f


def mlp(X_train,y_train,X_test,y_test):
    mlp_gs = MLPClassifier(max_iter=1000)
    parameter_space = {
        'hidden_layer_sizes': [(5,10,5),(10,)],
        'activation': ['tanh', 'relu'],
        'solver': ['sgd', 'adam'],
        'alpha': [0.0001, 0.05],
        'learning_rate': ['constant','adaptive'],
    }
    from sklearn.model_selection import GridSearchCV
    clf = GridSearchCV(mlp_gs, parameter_space, n_jobs=-1, cv=5)
    clf.fit(X_train, y_train) 
    predicted=clf.predict(X_test)
    a=accuracy_score(y_test, predicted)
    p=precision_score(y_test,predicted,average="macro")
    r=recall_score(y_test,predicted,average="macro")
    f=f1_score(y_test,predicted,average="macro")
    return a,p,r,f



def main(sir_file,cent_file,ml_model):
    l=[2,5,10,20,30,40,50,60,70,80,90,100]
    dic={}
    prec={}
    rec={}
    f1={}
    for i in l:
        df=pd.read_csv(sir_file)
        df['SIR_IC']=(df['infSIR']+df['infIC'])/2
        df1=df.drop(['infIC','infIC'],axis=1)
        def labelling(x,N):
            minscale=df1['SIR_IC'].min()
            maxscale=df1['SIR_IC'].max()
            rang=(maxscale-minscale)/N
            label=math.floor((x-minscale)//rang)+1
            return label
        df1['Label'] =df1.apply(lambda row:labelling(row['SIR_IC'],i),axis=1)
        df1.rename(columns = {'vertex':'Node'}, inplace = True)
        unpickled_data = pd.read_csv(cent_file)
        df3 = pd.merge(unpickled_data, df1)
        cols=['DC','CC','KC','LRAD','LRCC','LRKC']
        df3[cols] = df3[cols].apply(lambda x: (x/len(df3)),axis=0)
        df3=df3.drop(['Node','infSIR','beta'],axis=1)
        # get the locations
        X = df3.iloc[:, :-1]
        y = df3.iloc[:, -1]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30,random_state=0)
        a,p,r,f=ml_model(X_train,y_train,X_test,y_test)
        dic[i]=a
        prec[i]=p
        rec[i]=r
        f1[i]=f
    return dic,prec,rec,f1

l1=[logis_reg,mlp,decis_tree,rand_forest,svm,knn,navie_bayes]
l2=['logis_reg','mlp','decis_tree','rand_forest','svm','knn','navie_bayes']

for i in range(0,len(l1)):
    d1,d2,d3,d4=main("siricforall(web_0.2).csv","centraltiesallverticetest_web.csv",l1[i])
    new_list1 = list(d1.keys())
    new_list2 = list(d1.values())
    new_list3=list(d2.values())
    new_list4=list(d3.values())
    new_list5=list(d4.values())
    df=pd.DataFrame(list(zip(new_list1,new_list2,new_list3,new_list4,new_list5)),columns=['Labels','Accuracy','Precision','Recall','F1-score'])
    df.to_csv('{}.csv'.format(l2[i]))


C:\Users\SRMAP\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
C:\Users\SRMAP\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
C:\Users\SRMAP\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
C:\Users\SRMAP\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
1440 fits failed out of a total of 2880.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
------------------------------------------------

C:\Users\SRMAP\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
1440 fits failed out of a total of 2880.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
240 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\SRMAP\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\SRMAP\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\SRMAP\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 71, in _check_solver
    raise ValueError("

C:\Users\SRMAP\anaconda3\lib\site-packages\sklearn\utils\optimize.py:210: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn(
C:\Users\SRMAP\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
C:\Users\SRMAP\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
C:\Users\SRMAP\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
C:\Users\SRMAP\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
1440 fits failed out of a total of 2880.
The score on these train-test partitions for these parameters will be set to nan.
If these f

C:\Users\SRMAP\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
1440 fits failed out of a total of 2880.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
240 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\SRMAP\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\SRMAP\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\SRMAP\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 71, in _check_solver
    raise ValueError("

C:\Users\SRMAP\anaconda3\lib\site-packages\sklearn\utils\optimize.py:210: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn(
C:\Users\SRMAP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\SRMAP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\SRMAP\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
C:\Users\SRMAP\anaconda3\lib\site

C:\Users\SRMAP\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
1440 fits failed out of a total of 2880.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
240 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\SRMAP\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\SRMAP\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\SRMAP\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 71, in _check_solver
    raise ValueError("

C:\Users\SRMAP\anaconda3\lib\site-packages\sklearn\utils\optimize.py:210: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  warnings.warn(
C:\Users\SRMAP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\SRMAP\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\SRMAP\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
C:\Users\SRMAP\anaconda3\lib\site